In [36]:
import openmeteo_requests
import pandas as pd
import requests_cache
from datetime import datetime, timedelta
from retry_requests import retry

In [37]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [38]:
city_data_path = "../data/city_data.csv"
city_data = pd.read_csv(city_data_path)

In [39]:
url = "https://archive-api.open-meteo.com/v1/archive"
res_list = []

city_index = 0
for city in city_data["city"]:
    params = {
        "latitude": city_data["lat"][city_index],
        "longitude": city_data["lon"][city_index],
        "start_date": "2020-01-01",
        "end_date": datetime.strftime(datetime.now() - timedelta(3), "%Y-%m-%d"),
        "hourly": "temperature_2m",
        "daily": "temperature_2m_mean",
        "temperature_unit": "fahrenheit",
        "timezone": "auto",
    }
    res_list.append(openmeteo.weather_api(url, params=params))
    city_index += 1

city_index = 0
data_list = []
for res in res_list:
    hourly = res[0].Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )
    }
    hourly_data["temperature_2m"] = hourly_temperature_2m

    hourly_dataframe = pd.DataFrame(data=hourly_data)

    daily = res[0].Daily()
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()

    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s"),
            end=pd.to_datetime(daily.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left",
        )
    }
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["city"] = city_data["city"][city_index]
    
    daily_dataframe = pd.DataFrame(data=daily_data)

    data_list.append(daily_dataframe)

    city_index += 1

data_con = pd.concat(data_list)

data_merge = data_con.merge(city_data, how="inner", on="city")

In [ ]:
data_merge.to_csv("../data/weather_data.csv", index=False)